In [18]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [19]:
import pandas as pd
import numpy as np

In [20]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id="58ffefa3d697479f9cf66689c248dc81", client_secret="1a3406a8e3424e9c93533a086b202e38")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [21]:
def give_audio_features(playlist_link):
  playlist_URI = playlist_link.split("/")[-1].split("?")[0]
  track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

  # Initialize lists to store extracted information
  track_uris = []
  track_names = []
  artist_uris = []
  artist_names = []
  artist_popularities = []
  artist_genres_list = []
  albums = []
  track_popularities = []

  # Loop through each track in the playlist
  for track in sp.playlist_tracks(playlist_URI)["items"]:
      # URI
      track_uris.append(track["track"]["uri"])

      # Track name
      track_names.append(track["track"]["name"])

      # Main Artist
      main_artist = track["track"]["artists"][0]
      artist_uris.append(main_artist.get("uri", None))
      artist_names.append(main_artist.get("name", None))

      # If there's artist info available
      if artist_uris[-1]:
          artist_info = sp.artist(artist_uris[-1])
          # Popularity
          artist_popularities.append(artist_info.get("popularity", None))
          # Genres
          artist_genres_list.append(artist_info.get("genres", None))
      else:
          artist_popularities.append(None)
          artist_genres_list.append(None)

      # Album
      albums.append(track["track"]["album"].get("name", None))

      # Popularity of the track
      track_popularities.append(track["track"].get("popularity", None))

  # Create DataFrame
  df = pd.DataFrame({
      "Track URI": track_uris,
      "Track Name": track_names,
      "Artist URI": artist_uris,
      "Artist Name": artist_names,
      "Artist Popularity": artist_popularities,
      "Artist Genres": artist_genres_list,
      "Album": albums,
      "Track Popularity": track_popularities
  })

    # Initialize lists to store audio features
  audio_features_list = []

  # Loop through each track URI in your playlist
  for uri in track_uris:
      # Call sp.audio_features() to retrieve audio features for the track
      audio_features = sp.audio_features(uri)

      # Append the audio features to the list
      audio_features_list.append(audio_features[0])  # audio_features is a list, we only need the first (and usually only) element

  # Create a DataFrame from the list of audio features
  audio_features_df = pd.DataFrame(audio_features_list)


  final_df=pd.concat([df, audio_features_df], axis=1)

  return final_df

In [22]:
df1=give_audio_features("https://open.spotify.com/playlist/2sL5TXftq45m031SB0E0bR")

In [23]:
df2=give_audio_features("https://open.spotify.com/playlist/6x0cKmHY3mrv11j4WHOqhv")

In [24]:
df3=give_audio_features("https://open.spotify.com/playlist/3yVwlYvP9eQ5QTwEiJoDQf")

In [25]:
df4=give_audio_features("https://open.spotify.com/playlist/5vhAGH0Ba6Su9H9yBFOS1W")

In [26]:
df5=give_audio_features("https://open.spotify.com/playlist/4bhDmXHrCvufAAFaCATKIl")

In [27]:
df6=give_audio_features("https://open.spotify.com/playlist/5cuZ6XiI6Qs092g0CNZIH6")

In [28]:
df7=give_audio_features("https://open.spotify.com/playlist/4dyBWwqN11WS8zUCu6njgp")

In [29]:
df8=give_audio_features("https://open.spotify.com/playlist/5ZcyfzOYKggoBTuhmH8eKx")

In [30]:
final_df=pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=0)

In [31]:
final_df.shape

(758, 26)

In [32]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 758 entries, 0 to 92
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Track URI          758 non-null    object 
 1   Track Name         758 non-null    object 
 2   Artist URI         758 non-null    object 
 3   Artist Name        758 non-null    object 
 4   Artist Popularity  758 non-null    int64  
 5   Artist Genres      758 non-null    object 
 6   Album              758 non-null    object 
 7   Track Popularity   758 non-null    int64  
 8   danceability       758 non-null    float64
 9   energy             758 non-null    float64
 10  key                758 non-null    int64  
 11  loudness           758 non-null    float64
 12  mode               758 non-null    int64  
 13  speechiness        758 non-null    float64
 14  acousticness       758 non-null    float64
 15  instrumentalness   758 non-null    float64
 16  liveness           758 non-null 

In [33]:
final_df['mood']='happy'

In [34]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 758 entries, 0 to 92
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Track URI          758 non-null    object 
 1   Track Name         758 non-null    object 
 2   Artist URI         758 non-null    object 
 3   Artist Name        758 non-null    object 
 4   Artist Popularity  758 non-null    int64  
 5   Artist Genres      758 non-null    object 
 6   Album              758 non-null    object 
 7   Track Popularity   758 non-null    int64  
 8   danceability       758 non-null    float64
 9   energy             758 non-null    float64
 10  key                758 non-null    int64  
 11  loudness           758 non-null    float64
 12  mode               758 non-null    int64  
 13  speechiness        758 non-null    float64
 14  acousticness       758 non-null    float64
 15  instrumentalness   758 non-null    float64
 16  liveness           758 non-null 

In [35]:
final_df.to_csv("happy_training.csv")

In [36]:
calm_df=pd.read_csv("calm_training.csv")
happy_df=pd.read_csv("happy_training.csv")
energetic_df=pd.read_csv("energetic_training.csv")
sad_df=pd.read_csv("sad_training.csv")

In [37]:
final_training_df=pd.concat([calm_df, happy_df, energetic_df, sad_df], axis=0)

In [39]:
final_training_df.to_csv("final_training.csv")

In [40]:
train_df=pd.read_csv("final_training.csv")

In [42]:
train_df.columns

Index(['Track URI', 'Track Name', 'Album', 'Track Popularity', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'mood'],
      dtype='object')

In [59]:
train_df[train_df.duplicated('uri', keep=False) == True]

,Track URI,Track Name,Album,Track Popularity,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,mood
1,spotify:track:2qxmye6gAegTMjLKEBoR3d,Let Me Down Slowly,Narrated For You,86,0.652,0.5570,1,-5.714,0,0.0318,...,0.483,150.073,audio_features,2qxmye6gAegTMjLKEBoR3d,spotify:track:2qxmye6gAegTMjLKEBoR3d,https://api.spotify.com/v1/tracks/2qxmye6gAegT...,https://api.spotify.com/v1/audio-analysis/2qxm...,169354,4,calm
3,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,Photograph,x (Deluxe Edition),85,0.614,0.3790,4,-10.480,1,0.0476,...,0.201,107.989,audio_features,1HNkqx9Ahdgi1Ixy2xkKkL,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi...,https://api.spotify.com/v1/audio-analysis/1HNk...,258987,4,calm
4,spotify:track:3U4isOIWM3VvDubwSI3y7a,All of Me,Love In The Future (Expanded Edition),86,0.422,0.2640,8,-7.064,1,0.0322,...,0.331,119.930,audio_features,3U4isOIWM3VvDubwSI3y7a,spotify:track:3U4isOIWM3VvDubwSI3y7a,https://api.spotify.com/v1/tracks/3U4isOIWM3Vv...,https://api.spotify.com/v1/audio-analysis/3U4i...,269560,4,calm
8,spotify:track:5buL1qakN98gbmxYpYMmgU,Stay,Stay,49,0.693,0.7010,7,-8.632,0,0.0346,...,0.448,123.983,audio_features,5buL1qakN98gbmxYpYMmgU,spotify:track:5buL1qakN98gbmxYpYMmgU,https://api.spotify.com/v1/tracks/5buL1qakN98g...,https://api.spotify.com/v1/audio-analysis/5buL...,144204,4,calm
9,spotify:track:0TK2YIli7K1leLovkQiNik,Señorita,Señorita,78,0.759,0.5400,9,-6.039,0,0.0287,...,0.750,116.947,audio_features,0TK2YIli7K1leLovkQiNik,spotify:track:0TK2YIli7K1leLovkQiNik,https://api.spotify.com/v1/tracks/0TK2YIli7K1l...,https://api.spotify.com/v1/audio-analysis/0TK2...,190960,4,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,spotify:track:251PNRmJU9KcUnFQAB5t6I,Kal Ho Naa Ho,Kal Ho Naa Ho (Original Motion Picture Soundtr...,67,0.532,0.5290,0,-8.402,1,0.0352,...,0.295,81.989,audio_features,251PNRmJU9KcUnFQAB5t6I,spotify:track:251PNRmJU9KcUnFQAB5t6I,https://api.spotify.com/v1/tracks/251PNRmJU9Kc...,https://api.spotify.com/v1/audio-analysis/251P...,321240,4,sad
2967,spotify:track:54XrOdhUS2311wIt9m0OlD,Riha,Riha,64,0.395,0.1480,3,-11.016,1,0.0300,...,0.400,142.691,audio_features,54XrOdhUS2311wIt9m0OlD,spotify:track:54XrOdhUS2311wIt9m0OlD,https://api.spotify.com/v1/tracks/54XrOdhUS231...,https://api.spotify.com/v1/audio-analysis/54Xr...,287312,4,sad
2974,spotify:track:7pWAX5kJBULFTDNiH8oB42,Akela Hoon,Akela Hoon,49,0.508,0.0901,7,-22.456,1,0.0295,...,0.110,70.198,audio_features,7pWAX5kJBULFTDNiH8oB42,spotify:track:7pWAX5kJBULFTDNiH8oB42,https://api.spotify.com/v1/tracks/7pWAX5kJBULF...,https://api.spotify.com/v1/audio-analysis/7pWA...,224601,4,sad
2982,spotify:track:7uhINGViZPygI2AljxO8KN,cold/mess,cold/mess,57,0.509,0.4770,11,-6.571,1,0.0262,...,0.283,146.031,audio_features,7uhINGViZPygI2AljxO8KN,spotify:track:7uhINGViZPygI2AljxO8KN,https://api.spotify.com/v1/tracks/7uhINGViZPyg...,https://api.spotify.com/v1/audio-analysis/7uhI...,281219,3,sad


In [62]:
train_df=train_df.drop_duplicates(subset='Track URI', keep="last")

In [63]:
train_df

,Track URI,Track Name,Album,Track Popularity,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,mood
0,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),HEROES & VILLAINS,88,0.715,0.620,1,-6.005,0,0.0484,...,0.172,97.950,audio_features,2dHHgzDwk4BJdRwy9uXhTO,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,https://api.spotify.com/v1/tracks/2dHHgzDwk4BJ...,https://api.spotify.com/v1/audio-analysis/2dHH...,221520,4,calm
2,spotify:track:6GmUVqe73u5YRfUUynZK6I,Let Her Go,All the Little Lights,0,0.479,0.545,7,-7.346,1,0.0688,...,0.244,74.897,audio_features,6GmUVqe73u5YRfUUynZK6I,spotify:track:6GmUVqe73u5YRfUUynZK6I,https://api.spotify.com/v1/tracks/6GmUVqe73u5Y...,https://api.spotify.com/v1/audio-analysis/6GmU...,252733,4,calm
5,spotify:track:2b8fOow8UzyDFAE27YhOZM,Memories,Memories,28,0.764,0.320,11,-7.209,1,0.0546,...,0.575,91.019,audio_features,2b8fOow8UzyDFAE27YhOZM,spotify:track:2b8fOow8UzyDFAE27YhOZM,https://api.spotify.com/v1/tracks/2b8fOow8UzyD...,https://api.spotify.com/v1/audio-analysis/2b8f...,189486,4,calm
6,spotify:track:25eUoeaRmrhl82hllLHLau,Miracle,Miracle,56,0.635,0.700,1,-7.426,0,0.0423,...,0.161,120.081,audio_features,25eUoeaRmrhl82hllLHLau,spotify:track:25eUoeaRmrhl82hllLHLau,https://api.spotify.com/v1/tracks/25eUoeaRmrhl...,https://api.spotify.com/v1/audio-analysis/25eU...,165000,4,calm
7,spotify:track:4HBZA5flZLE435QTztThqH,Stuck with U (with Justin Bieber),Stuck with U,80,0.597,0.450,8,-6.658,1,0.0418,...,0.537,178.765,audio_features,4HBZA5flZLE435QTztThqH,spotify:track:4HBZA5flZLE435QTztThqH,https://api.spotify.com/v1/tracks/4HBZA5flZLE4...,https://api.spotify.com/v1/audio-analysis/4HBZ...,228482,3,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,spotify:track:1Al9f0Dpm7b2pTl1iglDob,Kitna Tha Roka,Kitna Tha Roka,33,0.449,0.265,4,-8.985,0,0.0315,...,0.251,117.797,audio_features,1Al9f0Dpm7b2pTl1iglDob,spotify:track:1Al9f0Dpm7b2pTl1iglDob,https://api.spotify.com/v1/tracks/1Al9f0Dpm7b2...,https://api.spotify.com/v1/audio-analysis/1Al9...,199000,3,sad
3008,spotify:track:0nHm0hPOwDYwwCbZUs0XKv,Am I Overthinking This?,Am I Overthinking This?,33,0.762,0.135,7,-17.362,1,0.0334,...,0.435,98.034,audio_features,0nHm0hPOwDYwwCbZUs0XKv,spotify:track:0nHm0hPOwDYwwCbZUs0XKv,https://api.spotify.com/v1/tracks/0nHm0hPOwDYw...,https://api.spotify.com/v1/audio-analysis/0nHm...,167166,4,sad
3009,spotify:track:3TvuDDztSve7iSxnd0bvHg,Aankhon Se Batana (feat. Dikshant) - Heartbrea...,Aankhon Se Batana (feat. Dikshant) [Heartbreak...,42,0.638,0.337,10,-10.085,1,0.0270,...,0.445,88.967,audio_features,3TvuDDztSve7iSxnd0bvHg,spotify:track:3TvuDDztSve7iSxnd0bvHg,https://api.spotify.com/v1/tracks/3TvuDDztSve7...,https://api.spotify.com/v1/audio-analysis/3Tvu...,204944,4,sad
3010,spotify:track:06LCamFUOtImIKi9mFRKiD,"Tum Kya Mile (From ""Rocky Aur Rani Kii Prem Ka...","Tum Kya Mile (From ""Rocky Aur Rani Kii Prem Ka...",76,0.271,0.726,7,-6.504,1,0.0527,...,0.532,87.425,audio_features,06LCamFUOtImIKi9mFRKiD,spotify:track:06LCamFUOtImIKi9mFRKiD,https://api.spotify.com/v1/tracks/06LCamFUOtIm...,https://api.spotify.com/v1/audio-analysis/06LC...,277500,4,sad
